# [Day 1: Report Repair](https://adventofcode.com/2020/day/1)

## Part 1

Given an expends report with an expense (integer) on each line, find two expenses that sum to 2020 and return their product.

Example:

`1721`<br>
`979`<br>
`366`<br>
`299`<br>
`675`<br>
`1456`

`1721 + 299 = 2020,`so return`514579`.

## Part 2

Find three expenses that satisfy the criteria above.

In [176]:
from typing import Optional, Set

def day1_part1(expenses: Set[int]) -> Optional[int]:
    for expense in expenses:
        if 2020 - expense in expenses:
            return expense * (2020 - expense)
    return None

def day1_part2(expenses: Set[int]) -> Optional[int]:
    for expense in expenses:
        for expense2 in expenses:
            if expense == expense2:
                pass
            expense3 = 2020 - (expense + expense2)
            if expense3 in expenses:
                return expense * expense2 * expense3
    return None

expense_report = frozenset(map(int, open('input/day1.txt', 'r').readlines()))
print('Part 1:', day1_part1(expense_report))
print('Part 2:', day1_part2(expense_report))

Part 1: 972576
Part 2: 199300880


# [Day 2: Password Philosophy](https://adventofcode.com/2020/day/2)

## Part 1

Given a list of password policies and passwords, return how many passwords are valid.

Format: `[min #]-[max #] [required letter]: [password]`

Example: `1-3 a: abcde`

In the example, the password must contain at least one and no more than three 'a's.

## Part 2

The numbers in the policy now refer to two 1-indexed positions in the password, exactly one of which must be the required letter.

In [177]:
from dataclasses import dataclass
from functools import reduce
import re
from typing import Tuple, List

@dataclass
class PasswordAndPolicy:
    range: Tuple[int, int]
    letter: str
    password: str

def parse_password(line: str) -> PasswordAndPolicy:
    groups = re.match('(\d+)-(\d+) ([a-z]): ([a-z]+)', line).groups()
    return PasswordAndPolicy((int(groups[0]), int(groups[1])), groups[2], groups[3])

def day2_part1(passwords: List[PasswordAndPolicy]) -> int:
    valid = 0
    for p in passwords:
        if p.range[0] <= reduce(lambda count, char: count + 1 if char == p.letter else count, p.password, 0) <= p.range[1]:
            valid += 1
    return valid

def day2_part2(passwords: List[PasswordAndPolicy]) -> int:
    valid = 0
    for p in passwords:
        if (p.password[p.range[0] - 1] == p.letter) ^ (p.password[p.range[1] - 1] == p.letter):
            valid += 1
    return valid

password_list = list(map(parse_password, open('input/day2.txt', 'r').readlines()))
print('Part 1:', day2_part1(password_list))
print('Part 2:', day2_part2(password_list))

Part 1: 467
Part 2: 441


# [Day 3: Toboggan Trajectory](https://adventofcode.com/2020/day/3)

## Part 1

Given a map of open squares (`.`) and trees (`#`), start at the top left, travel 3 characters to the right and 1 character down each step,
and count how many trees you hit until getting to the bottom row. The map repeats horizontally as many times as necessary.

## Part 2

Part 1 used a slope of $-1/3$. Try $-1$, $-1/3$, $-1/5$, $-1/7$, and $-2$ and multiply the results together.

In [178]:
def traverse(map_lines: List[str], right: int, down: int) -> int:
    trees = 0
    for index, line in enumerate(map_lines[::down]):
        if line[(index * right) % len(line)] == '#':
            trees += 1
    return trees

def day3_part1(map_lines: List[str]) -> int:
    return traverse(map_lines, 3, 1)

def day3_part2(map_lines: List[str]) -> int:
    return reduce(lambda x, y: x * y, map(lambda r_d: traverse(map_lines, r_d[0], r_d[1]), [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]))

map_input = open('input/day3.txt', 'r').read().splitlines()
print('Part 1:', day3_part1(map_input))
print('Part 2:', day3_part2(map_input))

Part 1: 153
Part 2: 2421944712


# [Day 4: Passport Processing](https://adventofcode.com/2020/day/4)

## Part 1

Valid passports have the following fields: `byr, iyr, eyr, hgt, hcl, ecl, pid, cid`. `cid` can be missing, but nothing else can.

Given a file with passports separated by empty lines, return the number of valid passports.

## Part 2

Field validation rules:

`byr: [1920, 2002]`<br>
`iyr: [2010, 2020]`<br>
`eyr: [2020, 2030]`<br>
`hgt: [150, 193]cm or [59, 76]in`<br>
`hcl: #[0-9]|[a-f]`<br>
`ecl: amb|blu|brn|gry|grn|hzl|oth`<br>
`pid: 9 digit number including leading zeroes`<br>
`cid: ignored`

In [179]:
@dataclass
class Passport:
    byr: int = None
    iyr: int = None
    eyr: int = None
    hgt: str = None
    hcl: str = None
    ecl: str = None
    pid: str = None

def parse_passport(passport: str) -> Passport:
    fields = dict(map(lambda x: x.split(':'), passport.split()))
    return Passport(int(fields.get('byr') or 0), int(fields.get('iyr') or 0), int(fields.get('eyr') or 0), fields.get('hgt') or "", fields.get('hcl') or "", fields.get('ecl') or "", fields.get('pid') or "")

def day4_part1(passports: List[Passport]) -> int:
    return sum(map(lambda p: all([p.byr, p.iyr, p.eyr, p.hgt, p.hcl, p.ecl, p.pid]), passports))

def day4_part2(passports: List[Passport]) -> int:
    def is_valid(passport: Passport) -> bool:
        height_re = re.match('(\d*)(cm|in)', passport.hgt)
        return all([passport.byr in range(1920, 2003),
                    passport.iyr in range(2010, 2021),
                    passport.eyr in range(2020, 2031),
                    height_re and ((height_re.group(2) == 'cm' and int(height_re.group(1)) in range(150, 194)) or (height_re.group(2) == 'in' and int(height_re.group(1)) in range(59, 77))),
                    re.match('#[0-9|a-f]{6}', passport.hcl),
                    re.match('amb|blu|brn|gry|grn|hzl|oth', passport.ecl),
                    re.match('\d{9}$', passport.pid)])
    return sum(map(is_valid, passports))

passport_list = list(map(parse_passport, open('input/day4.txt', 'r').read().split('\n\n')))
print('Part 1:', day4_part1(passport_list))
print('Part 2:', day4_part2(passport_list))

Part 1: 264
Part 2: 224


# [Day 5: Binary Boarding](https://adventofcode.com/2020/day/5)

## Part 1

Seats are numbered in binary partitioning, example `FBFBBFFRLR`. `F` and `B` are front and back half of rows, starting with
the range 0-127. `L` and `R` are left and right half of columns, starting with 0-7.

Seat ID is the row multiplied by 8 plus the column. Return the highest seat ID in the input.

## Part 2

Return the only missing seat ID, whose neighboring seats are in the input

In [180]:
def parse_seat(boarding_pass: str) -> Tuple[int, int]:
    return int(boarding_pass[:7].replace('F', '0').replace('B', '1'), 2), int(boarding_pass[7:].replace('L', '0').replace('R', '1'), 2)

def seat_id(seat: Tuple[int, int]) -> int:
    return seat[0] * 8 + seat[1]

def day5_part1(boarding_passes: List[str]) -> int:
    return max(map(seat_id, map(parse_seat, boarding_passes)))

def day5_part2(boarding_passes: List[str]) -> int:
    seats = sorted(map(seat_id, map(parse_seat, boarding_passes)))
    previous = seats[0] - 1
    for seat in seats:
        if seat - 1 != previous:
            return seat - 1
        previous = seat
    return -1

boarding_pass_list = open('input/day5.txt', 'r').readlines()
print('Part 1:', day5_part1(boarding_pass_list))
print('Part 2:', day5_part2(boarding_pass_list))

Part 1: 991
Part 2: 534
